# CS202: Compiler Construction

## In-class Exercises, Week of 02/27/2023

----

A4: Due in a week
Midterm: Next Friday (before spring break) - low pressure

2/27:
 - RCO
 - explicate control
 
3/1:
 - select instructions
 - register allocation
 
3/3:
 - everything else

# RCO

## Question 1

How do we handle `if` statements in rco?

We don't have to do much at all.

if is noit an operator, it's args dont need to be atomic.
 - call rco_exp on the condition
 - call rco_stmts on the then and else statements

----

# Explicate-Control

## Question 2

Describe the idea behind the explicate-control pass.

This should turn if stmts into control flow graphs (gotos/jumps). This doesn't produce x86 (that's select instructions)
The intermediate language this pass will produce is Cif.
 - Similar to poython (expr / stmts)
 - Different:
     - If statements must have gotos in then and else
     - Program is represented by a dict mapping labels to lists of stataments (a control flow graph)"
     
There are four pieces to consider when compiling an if statement:
 - the condition
 - the then branch (stmts)
 - the else branch (stmts)
 - the "rest of the stuff to do" after the if statement is done
     - this is called the "continuation"

We will always process statements backwards, so we always have a compiled version of the continuation.

## Question 3

Compile this program into a $C_{if}$ program, describing the input and output of explicate-control.

```
if 5 == 6:
    print(1)
else:
    print(0)
```

Control Flow Graph:
      l1: if 5 == 6
       /       \
      /         \
     /           \
 l2: Print(1)   l3: Print(0)
      \         /
       \       /
      l4: Return 0
      
                   cont = [Goto(l1)]
l1: if 5 == 6 cont cont = [If(cond, l2, l3)]
l2: Print(1) cont = [Goto(l4)]
l3: Print(0) cont = [Goto(l4)]
l4 becomes cont = [Return 0]

x = 5 [Assign(x,5), Assign(y,6), Assign(z,7), Return(0)]

y = 6 [Assign(y,6), Assign(z,7), Return(0)]

z = 7 [Assign(z,7), Return(0)]

(This started from bottom and went up)

label_2:

   - return 0
  
label_3:

   - print(1)
  
   - goto label_2
  
label_4:

   - print(0)
  
   - goto label_2
  
start:

   - tmp_1 = eq(5, 6)
  
   - if tmp_1: goto label_3 else: goto label_4

## Question 4

Describe the structure of `explicate_control`.

Start by following the grammar:

 - `ec_atm`
     - Constants => cif.Constants
     - Var => cif.Var
 - `ec_expr` compiles an expression into a Cin expression
     - Prim(op, args) => cif.Prim(op, new_args)
     - else call ec_atm
 - `ec_stmt` takes a statement and a continuation and returns a list of Cif statements
     - Assign(x,e) => [cif.Assign(x, ec_expr(e))] + cont
     - Print(e) => [cif.Print(ec_expr(e))] + cont
     - If (condition, then_stmts, else_stmts) => 
         - cond_label = create block for cont
         - then_label = create block for ec_stmts(then_stmts, [cif.Goto(cond_label)])
         - else_label = create block for ec_stmts(else_stmts, [cif.Goto(cond_label)])
         - return[cif(ec_expr(condition), cif.Goto(then_label), cif.Goto(else_label)]
         
 - `ec_stmts` takes a list of statements and a continuation, returns a list Cif statements
     - process list of statments in reverse
     - update current continuation by calling ec_stmt on each stmt and setting the continuation to whatever comes back
 - main body of the pass
     - start with continuation [cif.Return(0)]
     - call ec_stmts on statements of the program
     - set basic_blocks['start'] to the result
 - pass should have a global basic_blocks: Dict[str, List[cif.Stmt]]
 - pass should have a create_block gunction that adds a new block to basic_blocks with a unique name (using gensym)

----

# Select Instructions

## Question 5

Describe the new cases in `select_instructions`.

 - Mostly copy and paste.

 - Comparison case is really interesting

New cases:
 - `Assign(x, Prim(op, [a1, a2])` and `op` is a comparison primitive:
     - compile to 3 instructions:
         - cmpq a2, a1
         - set[cc] %al
             - cc is condition code for desired comparison op
             - eq(==): e, gt(>): g, gte(>=): ge, lt(<): l, lte(<=): le
         - movzbq %al, #x (moves byte reg into a 64-bit location)
 - If(condition, Goto(l1), Goto(l2))
    - cmpq condition, `$1`
    - je l1
    - jmp l2
 - Assign(x, Prim( 'not', [a1]))
    - movq al, #x
    - xorq $1, #x
 - Return(a1)
    - movq al, %rax
    - jmp conclusion
 - Goto(label)
    - jmp label

**NOTES NOT ANSWERS BELOW**

cmpq a2, a1 --compares, sets EFLAGS register

je label1 -- looking for equal
jge label -- looking for greater equal

what if we x = 5 == 6 though? how to get the result of comparison?

cmpq a2, a1
set e %al -- read write result of cmpq as boolean to al
movzbq %al, %r8  -- moves from byte register to regular register

----

# Register Allocation

## Question 6

Describe the liveness analysis for a control-flow graph.

 - Challenge is now that i have many blocks, and jumps between them; a varaiable might be read in a block I'm jumping to, and so it better remain live "across" jumps
 - Means I can't consider each block in isolation, i need to look at all bnlocks at once
 - treat things that might get read by the block I'm jumping to as things that are read by the jump instruction itself

## Question 7

Here is an x86 assembly program. Compute its live-after sets.

```
start:
  movq $42, #x
  cmpq $6, $5
  je label_2
  jmp label_3
label_2:
  addq $3, #x
  jmp conclusion
label_3:
  addq $4, #x
  jmp conclusion
```

When I find a jump instruction, I will compute the live-after sets for the target, and the live-before set for the target, and treat the live-before set as the variables read by the jump.

The `conclusion` block always has the empty live-before set.

```
start:                           {}
  movq $42, #x                   {x}
  cmpq $6, $5                       --> need to jump to label_2 --> {x}
  je label_2                     {} --> need to jump to label_3 --> {x}
  jmp label_3     <- start here: {}
label_2:                                    {x}
  addq $3, #x                               {}
  jmp conclusion                            {}
label_3:                              {x}
  addq $4, #x                         {}
  jmp conclusion                      {} <- empty live-before set
```

 - Global dict that remembers live-before sets for each block
 - Global dict that remembers list of live-after sets for each block

## Question 8

Describe the change to building the interference graph.

no change

## Question 9

Describe the change to graph coloring and assigning homes.

no change

----

# Patch Instructions

## Question 10

Describe the changes to patch_instructions.

the only change is to add a case for the `cmpq` instruction; it must not have an immediate as its second argument. If it does, you can use %rax as a temporary location to hold the immediate instead: `cmpq a1, a2` => `movq a2, %rax` then `cmpq a1, %rax`

cmpq instruction can't have immediate value as second argyument

----

# Prelude & Conclusion


## Question 11

Describe the changes to prelude_and_conclusion.

the "prelude" code now goes in the `main` block, with a jmp to the start block. The "conclusion" code now goes in the `conclusion` block.
 - Almost the same as assingment 3 except
   - Prelude goes in main
   - conclusion goes into conclusion block